In [1]:
#!pip install -q torch
#!pip install git+https://github.com/huggingface/transformers #huggingface transformers for downloading models weights
!pip install transformers
#!pip install -q datasets #huggingface datasets to download and manipulate datasets
!pip install  peft #Parameter efficient finetuning - for qLora Finetuning
#!pip install -q bitsandbytes #For Model weights quantisation
#!pip install -q trl #Transformer Reinforcement Learning - For Finetuning using Supervised Fine-tuning
#!pip install -q wandb -U #Used to monitor the model score during training

#!pip3 install -q git+https://github.com/casper-hansen/AutoAWQ

#!pip install auto-gptq 

  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/82/cc/bf022d6bc3996a5939c3ee39bde2b0e1f8bf6cea6ef9c9cdaf1639586237/peft-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.8 MB/s eta 0:00:00


In [2]:
!pip3 install  auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7
!pip3 install  optimum


Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 56.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 76.0 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for optimum from https://files.pythonhosted.org/packages/08/9e/de1fbdfa6a71cf3490d21a93942f48261208b5c00041947a526f3a26a8bd/optimum-1.15.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 24.6 MB/s eta 0:00:00


In [3]:
import json
#import re
#from pprint import pprint

import pandas as pd
import torch
#from datasets import Dataset, load_dataset
#from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    #BitsAndBytesConfig,
    #TrainingArguments,
    #pipeline,
    #logging,
)
#from trl import SFTTrainer # For supervised finetuning

In [8]:
per_device_eval_batch_size = 8

In [9]:
import math
from tqdm import tqdm
from torch.utils.data import Dataset

In [10]:
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, prompts, tokenizer):
        self.prompts = prompts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        return self.tokenizer(self.prompts[idx], padding=True, truncation=True, return_tensors='pt', max_length=1024)

In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [12]:
from torch.nn.utils.rnn import pad_sequence

In [13]:
def collate_fn(batch):
    # Pad inputs
    input_ids = pad_sequence([item['input_ids'].squeeze() for item in batch], batch_first=True)
    attention_mask = pad_sequence([item['attention_mask'].squeeze() for item in batch], batch_first=True)
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask}

In [14]:
def test_data(tokenizer, merged_model):
    
#     tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
#     tokenizer.pad_token = tokenizer.eos_token
#     tokenizer.padding_side = "right"
    
    prompts = list()
    references = list()
    ann0s = list()
    ann1s = list()
    ann2s = list()
    subjects = list()
    
    with open("/kaggle/input/mistral-7b/enron_lines/test.json", "r") as sequences:
        for sequence in sequences:
            sequence = json.loads(sequence)
            try:
                if len(sequence["text"]) < 1024:
                    prompt = sequence["text"]
                    subject =sequence["subject"]
                    ann0 = sequence["ann0"]
                    ann1 = sequence["ann1"]
                    ann2 = sequence["ann2"]
                    prompts.append(prompt)
                    ann0s.append(ann0)
                    ann1s.append(ann1)
                    ann2s.append(ann2)
                    subjects.append(subject)
                    references.append([subject, ann0, ann1, ann2])
            except ValueError:
                continue
    
    dataset = CustomDataset(prompts, tokenizer)
    data_loader = DataLoader(dataset, batch_size=per_device_eval_batch_size, num_workers=4, collate_fn=collate_fn)
    data_loader, merged_model = accelerator.prepare(data_loader, merged_model)
    
    
    tokenizer.padding_side='left'
   # n_batches = math.ceil(len(prompts)*1.0 / per_device_eval_batch_size)
    outputs = list()
    for prompts_batch_ids in tqdm(data_loader):
       # prompts_batch = prompts[i*per_device_eval_batch_size : (i+1)*per_device_eval_batch_size]
#         prompts_batch_ids = tokenizer(prompts_batch,
#             padding=True, truncation=True, return_tensors='pt').to('cuda')

        prompts_batch_ids = {k: v.to(accelerator.device) for k, v in prompts_batch_ids.items()}
        
        with accelerator.autocast():
            output_ids = merged_model.generate(
                **prompts_batch_ids, max_new_tokens=10,
                pad_token_id=tokenizer.pad_token_id, do_sample=True, top_p=0.9,temperature=0.5)
        
#         outputs_batch = [seq.split('[/INST]')[1] for seq in
#             tokenizer.batch_decode(output_ids.detach().cpu().numpy(), skip_special_tokens=True)]
        #print(outputs_batch)
        outputs_batch=[]
        for seq in tokenizer.batch_decode(output_ids.detach().cpu().numpy(), skip_special_tokens=True):
            print(seq)
            print("\n")
            print("------------------------------------------------------------")
            seq_val = seq.split('[/INST]')[1]
            outputs_batch.append(seq_val)
            #print(outputs_batch)
        outputs.extend(outputs_batch)
#     for prompt in prompts:
#         prompt = accelerator.prepare(prompt)
#         input_ids = tokenizer(prompt, return_tensors="pt", truncation=True)
#         output_ids = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)
#         output = tokenizer.batch_decode(output_ids.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
#         print(output)
#         outputs.extend(output)
    
    
        
        
    tokenizer.padding_side='right'
    
    df = pd.DataFrame({
        'prompt':prompts,
        'subjects':subjects,
        'ann0':ann0s,
        'ann1':ann1s,
        'ann2':ann2s,
        'model_outputs':outputs
    })

    df.to_csv('test_output.csv', index=False)


    bleu_score = bleu.compute(predictions=outputs, references=references)
    rouge_score = rouge.compute(predictions=outputs, references=references)
   # meteor_score = meteor.compute(predictions=outputs, references=references)

    return {
        'BLEU': round(bleu_score['bleu'], 4) * 100,
        'R1': round(rouge_score['rouge1'], 4) * 100,
        'R2': round(rouge_score['rouge2'], 4) * 100,
        'RL': round(rouge_score['rougeL'], 4) * 100,
        'RLsum': round(rouge_score['rougeLsum'], 4) * 100,
        #'meteor': round(meteor_score['meteor'], 4) * 100,

        #'METEOR': round(meteor_score['meteor'], 4) * 100
        }

In [15]:
model_path = '/kaggle/input/mistral-7b/merged_model'
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [17]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [20]:
merged_model= PeftModel.from_pretrained(base_model, '/kaggle/input/mistral-7b/mistralai-Email-Instruct')

In [ ]:
# prompt = "<s>[INST] Generate a subject for this email content, Thanks in advance for agreeing to speak at the Global Operations Controller Forum. There will be approximately 30 Enron business controllers present at the meeting. All have responsibility for mid and back office operations for the following Enron entities: Enron North America, Enron Europe, Enron South America, Enron Global Markets, Enron Industrial Markets, Enron Broadband Services and Enron Energy Services. Attendees will be here from Houston, Calgary, Tokyo, Sydney, London and New York (metals business). Attached for your reference is the agenda. There may be some slight changes before the forum begins, but this will give you a good idea of the topics to be covered and the other speakers who will address the group. You are scheduled to address the group as follows: [/INST] "

# input_ids = tokenizer(prompt, return_tensors="pt", truncation=False).input_ids.cuda()

# outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

# print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

In [ ]:
!nvidia-smi


In [21]:
from accelerate import Accelerator, DistributedType
accelerator = Accelerator()


In [22]:
test_data(tokenizer, merged_model)

  0%|          | 0/192 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
  1%|          | 1/192 [00:15<49:40, 15.61s/it]

[INST] Generate a subject for this email content, Hello Vince, We've now got some slick results on rigorous and replicable avg temp forecasting models for twelve major U.S. cities, with implications for weather derivative demand and supply (although they are not yet written up formally). Will you be in Phila anytime soon? I would like to sit down with you for a couple of hours to show you the results and discuss. Are you still interested in pursuing a joint project? I am excited about this. All the best, [/INST] n Weather Derivatives - Avg Temp


------------------------------------------------------------
[INST] Generate a subject for this email content, Congratulations on the birth of your daughter! Meagan called this afternoon to tell me the news. I will perhaps get more details when I get home tonight. I hope that Laura and the baby are doing well. Let us know what we can do to help. I am sure that Jackson is a proud brother. We will be anxious to meet our newest neighbor. --Sally 

  1%|          | 2/192 [00:27<42:35, 13.45s/it]

[INST] Generate a subject for this email content, Dear Enron Employees: I am saddened to inform you that there was an explosion and fire at our Teesside power facility in England at approximately 2:40 p.m. local time. At this point, we know that two employees have died, and at least two others have been seriously injured. The area has been evacuated and the power station is currently off-line. The cause of the incident is not yet known. Our hearts and deepest sympathies go out to the families of those affected by this terrible incident. Jeff Skilling [/INST]  08/10/04 Te


------------------------------------------------------------
[INST] Generate a subject for this email content, Joe: We are entering into an Agreement with Puget Sound Energy, Inc. in which we are agreeing to some chnages to our new form of Annex A. Enclosed is the new form which we have agreed to which should be attached to all trades done with Puget after February 2, 2000. Alos enclosed is the Agreement that we will

  2%|▏         | 3/192 [00:38<39:29, 12.54s/it]

...

 99%|█████████▉| 191/192 [42:09<00:13, 13.26s/it]

[INST] Generate a subject for this email content, As a result of restructuring out-of-the-money transactions with BP Capital and T. Boon Pickens, we have a total of $9,530,935 in the Schedule C for the NG Price book. These values are negative in Schedule C, thus they are positive value to the book. A total of $9,321,151 will be released out of Schedule C tonite (8/29/2000). The remaining $209,784 will be relesed by the end of the week. We are awaiting word on whether or not this value will be needed for credit reserve. Thanks, [/INST] ] 8/29/200


------------------------------------------------------------
[INST] Generate a subject for this email content, Deal excluded on Services desk on the original version and ST SW desk was adjusted differently. Please see me if you have any questions. Thanks [/INST]  \n Services desk exclusion. (2)


------------------------------------------------------------
[INST] Generate a subject for this email content, Mark; I have already spoken with Bret

100%|██████████| 192/192 [42:20<00:00, 13.23s/it]

[INST] Generate a subject for this email content, We have received an executed financial Master Agreement: Type of Contract: ISDA Master Agreement (Multicurrency-Cross Border) Effective Date: July 5, 2001 Enron Entity: Enron North America Corp. Counterparty: Doublewood XLIM Fund Ltd. (The) Transactions Covered: Approved for all products with the exception of: Foreign Exchange Confirming Entity: Enron North America Corp. Governing Law: New York Special Handling Requirements: (a) ENA is Calculation Agent, unless a Potential Event of Default or an Event of Default exists with respect to ENA, then Counterparty will act as Calculation Agent or will appoint a third party to act as such. (b) Counterparty has three (3) Local Business Days to accept or dispute a Confirmation. Copies will be distributed. [/INST] 2001-07-05


------------------------------------------------------------
[INST] Generate a subject for this email content, Gerald, Here is the Reciprocal Force Majeure language for you 

NameError: name 'bleu' is not defined

In [23]:
!pip install evaluate sacrebleu

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/0a/a6/2ac47e71e526bbcd97ea08f20d9ef7d3852e2594ec7b2d55f5d2bbfd7aae/sacrebleu-2.3.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.3 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 10.9 MB/s eta 0:00:00


In [26]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=0d4e14a8a7e4e6684f40e20462621ebdec6e867fa1c53ea8aaa92ea469e62017
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [24]:
import evaluate
def metric(data):
    #sub,ann0,ann1,ann2,gen_subject  = ["subject","ann0","ann1","ann2","gen_subject"]
    rouge = evaluate.load("rouge")
    meteor = evaluate.load("meteor")
    score = defaultdict(int)

    for _,x in tqdm(data.iterrows(),total=len(data)):

        #ref = ast.literal_eval(x["references"])
#         print(x["model_outputs"])
#         print("-----------------")



        rouge_s = rouge.compute(predictions=[x["model_outputs"]],references=[x["subjects"]])
        rouge_h = rouge.compute(predictions=[x["model_outputs"]] ,references=[[x["ann0"],x["ann1"],x["ann2"]]])

        score["rouge_1_sub"] += rouge_s['rouge1']
        score["rouge_2_sub"] += rouge_s['rouge2']
        score["rouge_L_sub"] += rouge_s['rougeL']

        score["rouge_1_human"] += rouge_h['rouge1']
        score["rouge_2_human"] += rouge_h['rouge2']
        score["rouge_L_human"] += rouge_h['rougeL']


        meteor_s = meteor.compute(predictions=[x["model_outputs"]],references=[x["subjects"]])
        meteor_h = meteor.compute(predictions=[x["model_outputs"]] ,references=[[x["ann0"], x["ann1"],x["ann2"]]])

        score["meteor_sub"] += meteor_s["meteor"]
        score["meteor_human"] += meteor_h["meteor"]

    print("The evalution scores are as follows:")

    for name,val in score.items():
        print(f"{name}: {val/len(data)}")

In [27]:
metric(pd.read_csv("test_output.csv"))

ImportError: cannot import name 'meteor_score' from 'nltk.translate' (/opt/conda/lib/python3.10/site-packages/nltk/translate/__init__.py)

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import ast


In [ ]:
torch.cuda.empty_cache() # PyTorch thing

In [ ]:
import gc

gc.collect()

